In [1]:
from transformers import DebertaV2Tokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data=pd.read_csv('dataset.csv')

In [90]:
train_texts, test_texts, train_labels, test_labels = train_test_split(data['User Input'], data['Polarity'], test_size=0.2, random_state=42)

In [91]:
model_name = "microsoft/deberta-v3-base"
tokenizer = DebertaV2Tokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3) # pos, neg, neutral

# Ensure padding token is defined
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [92]:
def preprocess_text(text):
    return text.lower().strip()

# Apply preprocessing
train_texts = train_texts.apply(preprocess_text)
test_texts = test_texts.apply(preprocess_text)

# Tokenize texts
train_texts_encodings = tokenizer(train_texts.tolist()[:2000], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
test_texts_encodings = tokenizer(test_texts.tolist()[:2000], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
# Map labels to integers
label_map = {"Positive": 0, "Negative": 1, "Neutral": 2}
train_labels = torch.tensor(train_labels.map(label_map).tolist()[:2000])
test_labels = torch.tensor(test_labels.map(label_map).tolist()[:2000])

In [93]:
train_dataset = TensorDataset(train_texts_encodings["input_ids"], train_labels)
test_dataset = TensorDataset(test_texts_encodings["input_ids"], test_labels)

# DataLoaders
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [109]:
def train_model(model, train_dataloader, num_epochs=7, learning_rate=2e-5, accumulation_steps=4):
    model.train()
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        epoch_loss = 0
        progress_bar = tqdm(train_dataloader, desc="Training")

        optimizer.zero_grad()  # Reset gradients at the beginning of each epoch
        for step, batch in enumerate(progress_bar):
            input_ids, labels = batch
            input_ids = input_ids.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
            epoch_loss += loss.item()

            # Backward pass
            loss.backward()

            # Update weights only after accumulating gradients
            if (step + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()  # Reset gradients after updating

            # Update the progress bar
            progress_bar.set_postfix(loss=loss.item())

        avg_epoch_loss = epoch_loss / len(train_dataloader)
        print(f"Average Epoch Loss: {avg_epoch_loss:.4f}")

    print("Training completed.")
train_model(model, train_dataloader, num_epochs=3, learning_rate=2e-5, accumulation_steps=4)

Epoch 1/3


Training:   0%|          | 0/125 [00:00<?, ?it/s]

Average Epoch Loss: 1.3500
Epoch 2/3


Training:   0%|          | 0/125 [00:00<?, ?it/s]

Average Epoch Loss: 1.1002
Epoch 3/3


Training:   0%|          | 0/125 [00:00<?, ?it/s]

Average Epoch Loss: 1.1038
Training completed.


In [112]:

def evaluate_model(model, test_dataloader):
    model.eval()  # Set the model to evaluation mode
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model.to(device)
    
    print("Evaluating the model")
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():  # Disable gradient calculation
        for batch in tqdm(test_dataloader, desc="Evaluating"):
            input_ids, labels = batch
            input_ids = input_ids.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(input_ids=input_ids)
            predictions = torch.argmax(outputs.logits, dim=-1)  # Get predicted class labels
            # Calculate the number of correct predictions
            # Create a DataFrame of input sentence, prediction, and actual label
            print(predictions,"hello")
            print(labels)
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)
    
    accuracy = correct_predictions / total_predictions  # Calculate accuracy
    print(f"Accuracy on the test set: {accuracy:.4f}")

# Call the evaluation function
evaluate_model(model, test_dataloader)



Evaluating the model


Evaluating:   0%|          | 0/125 [00:00<?, ?it/s]

tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([2, 2, 1, 0, 1, 1, 0, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([0, 2, 2, 2, 1, 2, 1, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([0, 0, 2, 0, 2, 2, 2, 0])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([0, 0, 1, 2, 1, 2, 0, 0])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([1, 2, 2, 0, 2, 0, 2, 0])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([2, 1, 2, 1, 0, 0, 1, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([0, 2, 1, 1, 1, 1, 2, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([0, 2, 1, 2, 0, 2, 2, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([1, 1, 1, 1, 2, 2, 1, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([2, 1, 2, 0, 1, 2, 2, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([2, 0, 0, 0, 0, 1, 0, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([1, 0, 1, 1, 0, 0, 1, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([2, 1, 1, 1, 2, 0, 1, 1])
tensor([2, 2, 2, 2, 2, 2, 2, 2]) hello
tensor([1, 1, 1, 1, 2, 2,

KeyboardInterrupt: 

In [105]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

In [106]:
def predict(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    predictions = []
    with torch.no_grad():  # Disable gradient calculation
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids, labels = batch
            input_ids = input_ids.to(device)

            # Forward pass
            outputs = model(input_ids=input_ids)
            preds = torch.argmax(outputs.logits, dim=-1)  # Get predicted class labels
            predictions.extend(preds.cpu().numpy())
    
    return predictions

# Predict on training set
train_predictions = predict(model, train_dataloader)

# Predict on test set
test_predictions = predict(model, test_dataloader)

Predicting:   0%|          | 0/125 [00:00<?, ?it/s]

Predicting:   0%|          | 0/125 [00:00<?, ?it/s]

In [108]:
print(train_predictions)
print(train_labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 